Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [19]:
batch_size = 128
hidden_units = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(hidden, weights2) + biases2
  # L2_W is the L2 Loss Regularization Strength, Lamda 
  L2_W = 1e-3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2_W * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 734.629761
Minibatch accuracy: 3.1%
Validation accuracy: 25.5%
Minibatch loss at step 500: 189.211212
Minibatch accuracy: 95.3%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 114.011581
Minibatch accuracy: 96.1%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 69.127052
Minibatch accuracy: 97.7%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 41.915207
Minibatch accuracy: 100.0%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 25.672058
Minibatch accuracy: 99.2%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 15.511297
Minibatch accuracy: 100.0%
Validation accuracy: 85.2%
Test accuracy: 91.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [34]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(hidden, weights2) + biases2
  # L2_W is the L2 Loss Regularization Strength, Lamda 
  L2_W = 1e-3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2_W * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (512 - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 640.250793
Minibatch accuracy: 11.7%
Validation accuracy: 17.4%
Minibatch loss at step 500: 191.198578
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Minibatch loss at step 1000: 115.953308
Minibatch accuracy: 100.0%
Validation accuracy: 71.7%
Minibatch loss at step 1500: 70.320786
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 2000: 42.646622
Minibatch accuracy: 100.0%
Validation accuracy: 71.7%
Minibatch loss at step 2500: 25.864027
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 3000: 15.687394
Minibatch accuracy: 100.0%
Validation accuracy: 72.0%
Test accuracy: 78.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  keep_prob = tf.placeholder(tf.float32)
  hidden_drop = tf.nn.dropout(hidden, keep_prob)
  logits = tf.matmul(hidden_drop, weights2) + biases2
  # L2_W is the L2 Loss Regularization Strength, Lamda 
  L2_W = 1e-3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2_W * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (512 - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 869.374390
Minibatch accuracy: 7.0%
Validation accuracy: 37.9%
Minibatch loss at step 500: 191.509476
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 116.190750
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 70.472130
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 42.741997
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 25.922594
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 3000: 15.722661
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Test accuracy: 83.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 256
hidden_units = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_units, hidden_units]))
  biases2 = tf.Variable(tf.zeros([hidden_units]))
  weights3 = tf.Variable(tf.truncated_normal([hidden_units, hidden_units]))
  biases3 = tf.Variable(tf.zeros([hidden_units]))
  weights4 = tf.Variable(tf.truncated_normal([hidden_units, num_labels]))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  keep_prob = tf.placeholder(tf.float32)
  hidden_drop1 = tf.nn.dropout(hidden1, keep_prob)
  hidden2 = tf.nn.relu(tf.matmul(hidden_drop1, weights2) + biases2)
  hidden_drop2 = tf.nn.dropout(hidden2, keep_prob)
  hidden3 = tf.nn.relu(tf.matmul(hidden_drop2, weights3) + biases3)
  hidden_drop3 = tf.nn.dropout(hidden3, keep_prob)
  logits = tf.nn.softmax(tf.matmul(hidden_drop3, weights4) + biases4)
  # L2_W is the L2 Loss Regularization Strength, Lamda 
  L2_W = 1e-3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +\
L2_W * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))
  
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 10000, 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(\
                            tf_train_dataset, weights1) + biases1), weights2) + biases2), weights3) + \
                                                        biases3), weights4) + biases4)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(\
                            tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + \
                                                        biases3), weights4) + biases4)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(\
                            tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + \
                                                        biases3), weights4) + biases4)

num_steps = 30001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 362.802582
Minibatch accuracy: 7.8%
Validation accuracy: 9.9%
Minibatch loss at step 1000: 135.422272
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 51.664204
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 20.721333
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 4000: 9.241048
Minibatch accuracy: 9.8%
Validation accuracy: 10.0%
Minibatch loss at step 5000: 4.530448
Minibatch accuracy: 55.1%
Validation accuracy: 57.4%
Minibatch loss at step 6000: 2.746995
Minibatch accuracy: 73.4%
Validation accuracy: 68.1%
Minibatch loss at step 7000: 2.134767
Minibatch accuracy: 75.8%
Validation accuracy: 76.9%
Minibatch loss at step 8000: 1.840727
Minibatch accuracy: 82.8%
Validation accuracy: 77.4%
Minibatch loss at step 9000: 1.745299
Minibatch accuracy: 84.4%
Validation accuracy: 77.8%
Minibatch loss at step 10000: 1.731079
Minibatch accuracy: 82.8%
Validation acc